In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import chromedriver_binary
from pymongo import MongoClient 
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
% matplotlib inline

In [2]:
# making connection to mongodb

client = MongoClient()
database_schoolFinds = client['db_schoolFinds']
collection_schoolsFindsPass = database_schoolFinds['collection_schoolFindsPass']
collection_schoolsFindsFail = database_schoolFinds['collection_schoolFindsFail']

In [3]:
driver = webdriver.Chrome()
driver.get("http://www.schoolfinds.com/")
driver.maximize_window()

# finding all the cities links

driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
sleep(3)

selectCityButton = driver.find_element_by_xpath("//div[@class='header-content visible-desktop']//div[@class='span4']//a[2]")
selectCityButton.click()
sleep(2)

allCityLinks = driver.find_elements_by_xpath("//div[@class='modal-body']//ul[@class='last']//a")

closeButton = driver.find_element_by_xpath('//*[@id="popup"]/footer/button')
closeButton.click()
sleep(2)

In [4]:
all_city_links_list = []

for eachCity in allCityLinks:
    all_city_links_list.append(eachCity.get_attribute('href'))
    
all_city_links_list = all_city_links_list[200:242]

In [5]:
all_city_links_list

['http://www.schoolfinds.com/bihar/patna.html',
 'http://www.schoolfinds.com/bihar/gaya.html',
 'http://www.schoolfinds.com/bihar/nawada.html',
 'http://www.schoolfinds.com/bihar/darbhanga.html',
 'http://www.schoolfinds.com/bihar/samastipur.html',
 'http://www.schoolfinds.com/bihar/nalanda.html',
 'http://www.schoolfinds.com/bihar/katihar.html',
 'http://www.schoolfinds.com/bihar/siwan.html',
 'http://www.schoolfinds.com/bihar/aurangabad.html',
 'http://www.schoolfinds.com/bihar/hajipur.html',
 'http://www.schoolfinds.com/bihar/banka.html',
 'http://www.schoolfinds.com/bihar/muzaffarpur.html',
 'http://www.schoolfinds.com/bihar/bhagalpur.html',
 'http://www.schoolfinds.com/bihar/champaran.html',
 'http://www.schoolfinds.com/himachal-pradesh/mandi.html',
 'http://www.schoolfinds.com/himachal-pradesh/kullu.html',
 'http://www.schoolfinds.com/himachal-pradesh/shimla.html',
 'http://www.schoolfinds.com/himachal-pradesh/solan.html',
 'http://www.schoolfinds.com/himachal-pradesh/kangra.html

In [6]:
for eachCity in all_city_links_list:
    
    
    print("working for URL ", eachCity)
    driver.get(eachCity)
    sleep(5)
    
    noOfSchoolsTotal = int(re.sub(r"\D", '', str(driver.find_element_by_xpath(r"//div[@class='product-count']").text)))  # 196
    print("Total no of schools are : ", noOfSchoolsTotal)
    
    noOfOlsEstimated = int(noOfSchoolsTotal/10)   # 19 suppose
    print("possible number of ols estimated : ", noOfOlsEstimated)
    
    print("scrolling to page end!!! : for ",noOfOlsEstimated, " times")
    
    for i in range(noOfOlsEstimated):   # cover ols created
        
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        sleep(3)
        
        
    driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
        
        
        
    if (noOfSchoolsTotal < 10):
        print("no of schools are less than 10")
        driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
    
        
        noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][1]" + "//li[contains(@class,'item')]"))
        print()
        print("No of lis  presentin this single ol :", noOfLisPresent_in_this_ol)
        
        for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"
            
            try:
                schoolAboutDict = {}

                schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(1) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                schoolAboutDict['schoolName'] = schoolName

                schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                schoolAboutDict['schoolAddress'] = schoolAddressRefined

                schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                schoolAboutDict['schoolEmail'] = schoolEmail

                schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                schoolAboutDict['schoolContact'] = schoolContact

                schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                schoolAboutDict['schoolURL'] = schoolURL
                
                if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                    print("Already Present in the db")
                    continue

                collection_schoolsFindsPass.insert_one(schoolAboutDict)
                sleep(2)
                
                print("School Details : ", schoolAboutDict)
                print()
                
            except:
                
                print("Error for the head url city : ", eachCity)
                
                schoolDictFail = {}
                schoolDictFail['Error City URL'] = str(eachCity)
                
                if len(str(schoolName) > 0):
                    schoolDictFail['Error School Name'] = str(schoolName)
                
                collection_schoolsFindsFail.insert_one(schoolDictFail)
                
                
        
    else:
        
        if (noOfSchoolsTotal > noOfOlsEstimated*10):   # 196 > 190 
            print("WE NEED AN EXTRAA ITERATION!!!!  AS  :", noOfSchoolsTotal," > " ,noOfOlsEstimated*10)
        
            for i in range(1, noOfOlsEstimated + 2):    # go by "ol" number 1,2,3,4,5,6,7,8,9,.....18,19,20

                print("-------------------------STARTING FOR OL---------------------------- : ", i)
                print()
                driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
                noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')]"))
                print()
                print("No of lis  presentin this ol :", noOfLisPresent_in_this_ol)

                for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"

                    try:
                        schoolAboutDict = {}

                        schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                        schoolAboutDict['schoolName'] = schoolName

                        schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                        schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                        schoolAboutDict['schoolAddress'] = schoolAddressRefined

                        schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                        schoolAboutDict['schoolEmail'] = schoolEmail

                        schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                        schoolAboutDict['schoolContact'] = schoolContact

                        schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                        schoolAboutDict['schoolURL'] = schoolURL

                        if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                            print("Already Present in the db")
                            continue


                        collection_schoolsFindsPass.insert_one(schoolAboutDict)
                        print("School Inserted with success")
                        sleep(2)

                        print("School Details : ", schoolAboutDict)
                        print()



                    except:

                        print("Error for the head url city : ", eachCity)

                        schoolDictFail = {}
                        schoolDictFail['Error City URL'] = str(eachCity)

                        if len(str(schoolName) > 0):
                            schoolDictFail['Error School Name'] = str(schoolName)

                        collection_schoolsFindsFail.insert_one(schoolDictFail)

        else:
            print("schools can be exact multiples of 10 ")
            
            for i in range(1, noOfOlsEstimated + 1):    # go by "ol" number 1,2,3,4,5,6,7,8,9,.....18,19

                print("-------------------------STARTING FOR OL---------------------------- : ", i)
                print()
                driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
                noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')]"))
                print()
                print("No of lis  presentin this ol :", noOfLisPresent_in_this_ol)

                for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"

                    try:
                        schoolAboutDict = {}

                        schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                        schoolAboutDict['schoolName'] = schoolName

                        schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                        schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                        schoolAboutDict['schoolAddress'] = schoolAddressRefined

                        schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                        schoolAboutDict['schoolEmail'] = schoolEmail

                        schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                        schoolAboutDict['schoolContact'] = schoolContact

                        schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                        schoolAboutDict['schoolURL'] = schoolURL

                        if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                            print("Already Present in the db")
                            continue


                        collection_schoolsFindsPass.insert_one(schoolAboutDict)
                        print("School Inserted with success")
                        sleep(2)

                        print("School Details : ", schoolAboutDict)
                        print()



                    except:

                        print("Error for the head url city : ", eachCity)

                        schoolDictFail = {}
                        schoolDictFail['Error City URL'] = str(eachCity)

                        if len(str(schoolName) > 0):
                            schoolDictFail['Error School Name'] = str(schoolName)

                        collection_schoolsFindsFail.insert_one(schoolDictFail)

        

working for URL  http://www.schoolfinds.com/bihar/patna.html
Total no of schools are :  62
possible number of ols estimated :  6
scrolling to page end!!! : for  6  times
WE NEED AN EXTRAA ITERATION!!!!  AS  : 62  >  60
-------------------------STARTING FOR OL---------------------------- :  1


No of lis  presentin this ol : 10
School Inserted with success
School Details :  {'schoolName': "ST. THOMAS' HIGH SCHOOL", 'schoolAddress': ': Chapaladebi Road Dass Nagar Howrah-711 105', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 033-65401744', 'schoolURL': 'http://www.schoolfinds.com/st-thomas-high-school-13933.html', '_id': ObjectId('5b8a5637edf2ee33a8fda940')}

School Inserted with success
School Details :  {'schoolName': 'R P S PUBLIC SCHOOL', 'schoolAddress': ': patna masaurhi road- p.s. agamkhan- p.o.b. pahari via gulzarbagh patna bihar -800007', 'schoolEmail': 'Email: ddhanwani@gmail.com', 'schoolContact': 'Contact Number: 0612 -351977', 'schoolURL': 'http://www.schoolfind

School Inserted with success
School Details :  {'schoolName': 'R.D. PUBLIC SCHOOL', 'schoolAddress': ': Vikram Vihar- Tekari- Gaya- Bihar -824236', 'schoolEmail': 'Email: rdpublicschool04@gmail.com', 'schoolContact': 'Contact Number: 9334227170', 'schoolURL': 'http://www.schoolfinds.com/r-d-public-school-9323.html', '_id': ObjectId('5b8a5683edf2ee33a8fda953')}

-------------------------STARTING FOR OL---------------------------- :  2


No of lis  presentin this ol : 0
-------------------------STARTING FOR OL---------------------------- :  3


No of lis  presentin this ol : 0
working for URL  http://www.schoolfinds.com/bihar/nawada.html
Total no of schools are :  5
possible number of ols estimated :  0
scrolling to page end!!! : for  0  times
no of schools are less than 10

No of lis  presentin this single ol : 5
School Details :  {'schoolName': 'JAWAHAR NAVODAYA VIDYALAYA', 'schoolAddress': ': REWAR P O PAKRIBARAWAN DISTT NAWADA BIHAR -805124', 'schoolEmail': 'Email:', 'schoolContact':

School Inserted with success
School Details :  {'schoolName': 'P C HIGH SCHOOL', 'schoolAddress': ': PATSA SAMASTIPUR BIHAR -848101', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 06275 -282122', 'schoolURL': 'http://www.schoolfinds.com/p-c-high-school.html', '_id': ObjectId('5b8a56e8edf2ee33a8fda967')}

School Inserted with success
School Details :  {'schoolName': 'KENDRIYA VIDYALAYA', 'schoolAddress': ': SAMASTIPUR E C RAILWAY SAMASTIPUR DISTT BIHAR -848101', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 527', 'schoolURL': 'http://www.schoolfinds.com/kendriya-vidyalaya-108979.html', '_id': ObjectId('5b8a56ebedf2ee33a8fda968')}

School Inserted with success
School Details :  {'schoolName': 'JAWAHAR NAVODAYA VIDYALAYA', 'schoolAddress': ': BIRAULI SAMASTIPUR BIHAR -848113', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 32281', 'schoolURL': 'http://www.schoolfinds.com/jawahar-navodaya-vidyalaya-75244.html', '_id': ObjectId('5b8a56ededf2ee33a8fda969

School Details :  {'schoolName': 'KENDRIYA VIDYALAYA', 'schoolAddress': ': KATIHAR BIHAR -854105', 'schoolEmail': 'Email: kvps_2002@yahoo.com', 'schoolContact': 'Contact Number: 08418 -2733', 'schoolURL': 'http://www.schoolfinds.com/kendriya-vidyalaya-108981.html', '_id': ObjectId('5b8a5751edf2ee33a8fda97c')}

working for URL  http://www.schoolfinds.com/bihar/siwan.html
Total no of schools are :  7
possible number of ols estimated :  0
scrolling to page end!!! : for  0  times
no of schools are less than 10

No of lis  presentin this single ol : 7
School Details :  {'schoolName': 'D A V CENTENARY PUBLIC SCHOOL', 'schoolAddress': ': KABIR MATH KANDHWARA SIWAN BIHAR -841226', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 22078', 'schoolURL': 'http://www.schoolfinds.com/d-a-v-centenary-public-school-12136.html', '_id': ObjectId('5b8a5764edf2ee33a8fda97d')}

School Details :  {'schoolName': 'EMMANUEL MISSION HIGH SCHOOL', 'schoolAddress': ': ANAND NAGAR SIWAN BIHAR -841226', 's

School Details :  {'schoolName': 'C S DAV PUBLIC SCHOOL', 'schoolAddress': ': KOTWA ROAD BANKAT MOTIHARI EAST CHAMPARAN BIHAR -845407', 'schoolEmail': 'Email: csdavps.motihari@gmail.com', 'schoolContact': 'Contact Number: 06252 -290406', 'schoolURL': 'http://www.schoolfinds.com/c-s-dav-public-school.html', '_id': ObjectId('5b8a57c7edf2ee33a8fda991')}

School Details :  {'schoolName': 'JAWAHAR NAVODAYA VIDYALAYA', 'schoolAddress': ': SHAKTI NAGAR CHANDAN DAM PO LAXMIPUR DISTT BANKA BIHAR -813102', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 03844 -32420', 'schoolURL': 'http://www.schoolfinds.com/jawahar-navodaya-vidyalaya-75242.html', '_id': ObjectId('5b8a57cbedf2ee33a8fda992')}

School Details :  {'schoolName': 'KENDRIYA VIDYALAYA', 'schoolAddress': ': PBS COLLEGE CAMPUS POST & DISTT BANKA BIHAR -813102', 'schoolEmail': 'Email: kv_masarak@yahoo.co.in', 'schoolContact': 'Contact Number: 06159 -224202', 'schoolURL': 'http://www.schoolfinds.com/kendriya-vidyalaya-108998.htm

...

School Inserted with success
School Details :  {'schoolName': 'GYAN BHARTI ACADEMY', 'schoolAddress': ': SWAHIDBAKERI TEJPUR SONITPUR ASSAM -784001', 'schoolEmail': 'Email: d_shahapurkar@hotmail.com', 'schoolContact': 'Contact Number:', 'schoolURL': 'http://www.schoolfinds.com/gyan-bharti-academy.html', '_id': ObjectId('5b8a5b88edf2ee33a8fdaa72')}

School Inserted with success
School Details :  {'schoolName': 'ASSAM RIFLES SCHOOL- CHARDUAR', 'schoolAddress': ': BALIPARA- CHARIDUAR- SONITPUR ASSAM PIN - 784 103 -784103', 'schoolEmail': 'Email: assamriflesschool11@gmail.com', 'schoolContact': 'Contact Number: 9436131820', 'schoolURL': 'http://www.schoolfinds.com/assam-rifles-school-charduar.html', '_id': ObjectId('5b8a5b8cedf2ee33a8fdaa73')}

-------------------------STARTING FOR OL---------------------------- :  2


No of lis  presentin this ol : 0
working for URL  http://www.schoolfinds.com/assam/guwahati.html
Total no of schools are :  36
possible number of ols estimated :  3
scrollin

School Inserted with success
School Details :  {'schoolName': 'JAWAHAR NAVODAYA VIDYALAYA', 'schoolAddress': ': WOMENS ITI CAMPUS PO- BORGURI DT- TINSUKIA ASSAM -786126', 'schoolEmail': 'Email: jnvtsk@rediffmail.com', 'schoolContact': 'Contact Number: 03759 -291738', 'schoolURL': 'http://www.schoolfinds.com/jawahar-navodaya-vidyalaya-75307.html', '_id': ObjectId('5b8a5bf1edf2ee33a8fdaa85')}

School Inserted with success
School Details :  {'schoolName': 'HOLY BROOK SCHOOL', 'schoolAddress': ': PAKHARIJIAN GAON NEAR ST LUKES HOSPITAL- TINSUKIA ASSAM -786146', 'schoolEmail': 'Email: shamimhazarika@yahoo.com', 'schoolContact': 'Contact Number: 3742338597', 'schoolURL': 'http://www.schoolfinds.com/holy-brook-school.html', '_id': ObjectId('5b8a5bf5edf2ee33a8fdaa86')}

School Inserted with success
School Details :  {'schoolName': 'HINDUSTANI KENDRIYA VIDYALAYA', 'schoolAddress': ': N.H.-37- Laipuli- Hijuguri- Tinsukia(Assam) -786125', 'schoolEmail': 'Email: singeshwarroy@yahoo.in', 'schoolCon

Total no of schools are :  49
possible number of ols estimated :  4
scrolling to page end!!! : for  4  times
WE NEED AN EXTRAA ITERATION!!!!  AS  : 49  >  40
-------------------------STARTING FOR OL---------------------------- :  1


No of lis  presentin this ol : 10
School Inserted with success
School Details :  {'schoolName': 'VIVEKANANDA VIDYA MANDIR', 'schoolAddress': ': SECTOR II JAGANNATH NAGAR H E C COLONY RANCHI JHARKHAND -834004', 'schoolEmail': 'Email: vvm.ranchi@gmail.com', 'schoolContact': 'Contact Number: 0651 -2441422', 'schoolURL': 'http://www.schoolfinds.com/vivekananda-vidya-mandir.html', '_id': ObjectId('5b8a5c6aedf2ee33a8fdaa98')}

School Inserted with success
School Details :  {'schoolName': 'RAM TAHAL CHOUDHARY HIGH SCHOOL', 'schoolAddress': ': PHED (KH TOLA) BOOTI RANCHI JHARKHAND -834009', 'schoolEmail': 'Email: sddavps_jamtara@rediffmail.com', 'schoolContact': 'Contact Number: 01676 -542114', 'schoolURL': 'http://www.schoolfinds.com/ram-tahal-choudhary-high-scho

School Inserted with success
School Details :  {'schoolName': 'JAMSHEDPUR PUBLIC SCHOOL', 'schoolAddress': ': PANCHAVATI ROAD NEW BARIDIH JAMSHEDPUR JHARKHAND -831017', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 2285405', 'schoolURL': 'http://www.schoolfinds.com/jamshedpur-public-school.html', '_id': ObjectId('5b8a5caeedf2ee33a8fdaaab')}

-------------------------STARTING FOR OL---------------------------- :  2


No of lis  presentin this ol : 0
working for URL  http://www.schoolfinds.com/jharkhand/hazaribagh.html
Total no of schools are :  29
possible number of ols estimated :  2
scrolling to page end!!! : for  2  times
WE NEED AN EXTRAA ITERATION!!!!  AS  : 29  >  20
-------------------------STARTING FOR OL---------------------------- :  1


No of lis  presentin this ol : 10
School Inserted with success
School Details :  {'schoolName': 'AGRASEN DAV PUB. SCHOOL', 'schoolAddress': ': P.O BHARECH NAGAR VIA RAMGARH CANTT DISTT HAZARIBAGH JHARKHAND -829146', 'schoolEmail':

School Inserted with success
School Details :  {'schoolName': 'KENDRIYA VIDYALAYA', 'schoolAddress': ': BHULI TOWNSHIP SRAMIK NAGAR DHANBAD JHARKHAND -828104', 'schoolEmail': 'Email: helpdesk@xeenet.xeedhn.xeemail.com', 'schoolContact': 'Contact Number: 2535', 'schoolURL': 'http://www.schoolfinds.com/kendriya-vidyalaya-109051.html', '_id': ObjectId('5b8a5d0cedf2ee33a8fdaabe')}

School Inserted with success
School Details :  {'schoolName': 'KENDRIYA VIDYALAYA', 'schoolAddress': ': KOYALANAGAR B C C L TOWNSHIP P O KOYALANAGAR DHANBAD JHARKHAND -826005', 'schoolEmail': 'Email: helpdesk@xeenet.xeedhn.xeemail.com', 'schoolContact': 'Contact Number: 5406', 'schoolURL': 'http://www.schoolfinds.com/kendriya-vidyalaya-109043.html', '_id': ObjectId('5b8a5d0fedf2ee33a8fdaabf')}

-------------------------STARTING FOR OL---------------------------- :  2


No of lis  presentin this ol : 0
-------------------------STARTING FOR OL---------------------------- :  3


No of lis  presentin this ol : 0
---